# IATA

### replace countries names

In [1]:
import pandas as pd

In [22]:
folder = r'C:\Users\Ensheng\Desktop\mapping\IATA\\'
in_table = folder + 'IATA_country_USairport_county_07_17.csv'
df = pd.read_csv(in_table)
print(len(df))
df.head(5)

370975


,year,originCountryId,originCountryName,destinationAirportName,destinationAirportCode,destinationAirportId,destination_state,destination_county,destination_countyDetailed,paxVolume
0,2007,73,United Kingdom,Naval Outlying Field Barin,NHX,3225,Alabama,Baldwin,Baldwin County,3
1,2007,101,Italy,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,1
2,2007,102,Jamaica,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,75
3,2007,103,Jordan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2
4,2007,104,Japan,Muscle Shoals Regional Airport,MSL,3210,Alabama,Colbert,Colbert County,2


In [23]:
df['originRegion'] = df['originCountryName'].replace({
    'Ã…land': 'Aland',
    'Antigua-Barbuda': 'Antigua and Barbuda',
    'Bahamas': 'The Bahamas',
    'Bonaire, Saint Eustatius and Saba': 'Caribbean Netherlands',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Brunei Darussalam': 'Brunei',
    'Central African Rep': 'Central African Republic',
    #'Christmas Island': '',
    'Congo (Brazzaville)': 'Republic of Congo',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Cote D\'Ivoire': 'Ivory Coast',
    'Curacao': 'Curacao',
    'Dominican Rep': 'Dominican Republic',
    #'French Guiana': '',
    #'Gibraltar': '',
    #'Guadeloupe': '',
    'Guinea-Bissau': 'Guinea Bissau',
    'Hong Kong (SAR)': 'Hong Kong S.A.R.',
    'Korea (North)': 'North Korea',
    'Korea (South)': 'South Korea',
    'Macao (SAR)': 'Macao S.A.R',
    #'Martinique': '',
    #'Mayotte': '',
    'Micronesia': 'Federated States of Micronesia',
    'Netherlands Antilles (obsolete)': 'Netherlands Antilles',
    'North Mariana Isl': 'Northern Mariana Islands',
    #'Reunion': '',
    'Sao Tome-Principe': 'Sao Tome and Principe',
    'Serbia': 'Republic of Serbia',
    'St Barthelemy': 'Saint Barthelemy',
    'St Kitts-Nevis': 'Saint Kitts and Nevis',
    'St Lucia': 'Saint Lucia',
    'St Maarten (Dutch Part)': 'Sint Maarten',
    'St Martin': 'Saint Martin',
    'St Pierre-Miquelon': 'Saint Pierre and Miquelon',
    'St Vincent-Grenad': 'Saint Vincent and the Grenadines',
    #'Svalbard': '',
    #'Timor-leste': '',
    'Trinidad-Tobago': 'Trinidad and Tobago',
    'Turks-Caicos': 'Turks and Caicos Islands',
    #'Tuvalu': '',
    'Viet Nam': 'Vietnam',
    'Virgin Islands (GB)': 'British Virgin Islands',
    'Virgin Islands (US)': 'United States Virgin Islands',
    'Wallis-Futuna Isl': 'Wallis and Futuna'})

In [7]:
in_table = folder + 'origin_XY.csv'
odf = pd.read_csv(in_table)
print(len(odf))
odf.head(5)

253


,Country,ISO,Origin_X,Origin_Y
0,Afghanistan,AFG,66.004734,33.835231
1,Aland,ALD,19.953288,60.214887
2,Albania,ALB,20.049834,41.142450
3,Algeria,DZA,2.617323,28.158938
4,American Samoa,ASM,-170.718026,-14.304460


In [32]:
# Note: remove IATA duplicates in the tables 954->932
in_table = folder + 'destination_XY.csv'
ddf = pd.read_csv(in_table)
print(len(ddf))
ddf.head(5)

932


,IATA,DESTINATION_X,DESTINATION_Y
0,0AK,-162.899579,61.934555
1,0B8,-72.031612,41.251316
2,0R3,-92.084228,29.975783
3,0S9,-122.810659,48.053815
4,16A,-162.440613,60.906011


In [33]:
result = pd.merge(df, odf, how='left', left_on='originRegion',right_on='Country')
len(result)

370975

In [34]:
result = pd.merge(result, ddf, how='left', left_on='destinationAirportCode',right_on='IATA')
len(result)

370975

In [35]:
# output as a csv table
output_csv = folder + 'flow_XY.csv'
result.to_csv(output_csv,index = False,encoding='utf-8')